In [1]:
import time
from multiprocessing import Process


def test(x):
    count  = 0
    for i in range(0, x**x):
      count += 1


if __name__ == "__main__":
    starttime = time.time()
    processlist = []
    for i in range(0, 4):
        process = Process(target=test, args=(9,))
        processlist.append(process)
        process.start()

    for process in processlist:
        process.join()
    endtime = time.time()
    print(f"Time taken {endtime-starttime} seconds")

Time taken 1.15525221824646 seconds


In [2]:
import time


def test(x):
    count = 0
    for i in range(0, x ** x):
        count += 1


starttime = time.time()
for i in range(0, 4):
    test(9)
endtime = time.time()
print(f"Time taken {endtime-starttime} seconds")

Time taken 121.42066097259521 seconds


In [3]:
import time
import threading


def test(x):
    count = 0
    for i in range(0, x ** x):
        count += 1


if __name__ == "__main__":
    starttime = time.time()
    threads = []
    for i in range(0, 4):
        thread = threading.Thread(target=test, args=(9,))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    endtime = time.time()
    print(f"Time taken {endtime-starttime} seconds")

Time taken 152.54156851768494 seconds
